<strong>Pychemkin</strong> provides a set of basic mixture utilities enabling the creation of new mixtures from the existing ones. The mixing method, as its name suggested, let you combine two mixtures at constant pressure with specific constraints. The <i><u>adiabaticmixing</u></i> method combines the two mixtures while keeping the overall enthalpy constant. The temperature of the combined mixture, in this case, is determined by the conservation of the overall enthalpy of the two "parent" mixtures. The <i><u>isothermalmixing</u></i>, on the other hand, simply combines the two mixtures and determines the composition of the final mixture according to the mole or mass ratios specified. You can assign any temperature value to the combined mixture since the enthalpy conservation is not considered by this method.

This tutorial will show you the usage of these two <em>Mixture</em> mixing methods, <i><u>isothermalmxing</u></i> and <i><u>adiabaticmixing</u></i>, and the difference in the temperature of the combined/final mixture between the two mixing methods. You will first create a fuel (CH4) mixture and an air (O2+N2) mixture and then make a fuel-air mixture by mixing them isothermally (i.e., without considering energy conservation) with a given air-to-fuel mass ratio. Afterwards, you will dilute the fuel-air mixture with argon (AR) adiabatically with the molar/volumetric ratio specified.   

In [ ]:
import os
import chemkin as ck                      # Chemkin

# check working directory
current_dir = os.getcwd()
print('current working directory: ' + current_dir)
# set verbose mode
ck.verbose = True

<h5>Set up the <em>Chemistry Set</em></h5>The first task of every <strong>PyChemkin</strong> project is to create and preprocess the <em>Chemistry</em> set object for the current PyChemkin project. Here the GRI 3.0 mechanism for methane combustion is used to instantiate the <code>MyGasMech</code> <em>Chemistry</em> object.

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = ck.ansys_dir + '\\reaction\\data'
mechanism_dir = data_dir
# create a chemistry set based on GRI 3.0
MyGasMech = ck.Chemistry(label='GRI 3.0')
# set mechanism input files
# inclusion of the full file path is recommended
MyGasMech.chemfile = mechanism_dir+'\\grimech30_chem.inp'
MyGasMech.thermfile = mechanism_dir+'\\grimech30_thermo.dat'
# transport data not needed
# preprocess the mechanism files
iError = MyGasMech.preprocess()

<h5>Set up the </em>Mixtures</em></h5>First create the <code>fuel</code> mixture consisting of 100% methane.

In [ ]:
# create the fuel mixture
# note: mixture pressures are not specified because pressure is not required for the calculations here
# the mixing process is assumed to take place at fixed pressure; i.e., the mixtures are at the same pressure
fuel = ck.Mixture(MyGasMech)
# set mole fraction
fuel.X = [('CH4', 1.0)]
fuel.temperature = 300.0

Create the <code>air</code> mixture at the same temperature as the fuel mixture.

In [ ]:
# create the air mixture
air = ck.Mixture(MyGasMech)
# set mole fraction
air.X = [('O2', 0.21), ('N2', 0.79)]
air.temperature = 300.0

<h5>Specify the mixing ratio</h5>Specify the mixing formula in mass ratio of <code>fuel</code>:<code>air</code>=1.00:17.19. 

In [ ]:
# mix the fuel and the air with an air-fuel ratio of 17.19 (almost stoichiometric?)
mixture_recipe = [(fuel, 1.0), (air, 17.19)]

<h5>Combine the two mixtures</h5>Use the <em>Mixture</em> method <code>isothermalmixing</code> to form the new <code>premixed</code> mixture. Set <code>mode='mass'</code> to indicate the mixing formula <code>mixture_recipe</code> is in mass ratio. Also, you need to specify the temperature of the final mixture <code>premixed</code> for "isothermal" mixing. 

In [ ]:
# create the new mixture (the air-fuel ratio is by mass)
premixed = ck.isothermalmixing(recipe=mixture_recipe, mode='mass', finaltemperature=300.0)

List the molar composition of the final <code>premixed</code> mixture for verification. The molar composition should resemble the stoichiometric methane-air mixture.

In [ ]:
# list molar composition
premixed.listcomposition(mode='mole')

<h5>Set up another <em>Mixture</em></h5>Create the argon diluant mixture <code>ar</code> with a different mixture temperature at 600K.

In [ ]:
# now create an argon mixture
ar = ck.Mixture(MyGasMech)
# species composition
ar.X = [('AR', 1.0)]
# mixture temperature
ar.temperature = 600.0

<h5>Define the mixing ratio</h5>Set the mixing "recipe" in volume ratio.

In [ ]:
# dilute the premixed mixture adiabatically with the ar mixture by 30% by volume
dilute_recipe = [(premixed, 0.7), (ar, 0.3)]

<h5>Add <code>ar</code> to the <code>premixed</code> mixture</h5>Create the <code>diluted</code> mixture by using the <em>Mixture</em> method <code>adiabaticmixing</code>. The temperature of the new <code>diluted</code> mixture is dtermined by enthalpy conservation so you do not need to provide the final temperature here. Set <code>mode='mole'</code> to indicate the mixing formula <code>dilute_recipe</code> is in molar ratio.  

In [ ]:
# create the diluted mixture
diluted = ck.adiabaticmixing(recipe=dilute_recipe, mode='mole')

List the molar composition of the final <code>diluted</code> mixture for verification.

In [ ]:
# list molar composition
diluted.listcomposition(mode='mole')

List the temperatures of the three mixtures, <code>premixed</code>, <code>ar</code>, and <code>diluted</code>, in the dilution step for comparison. The temperature of the <code>diluted</code> mixture should be in between those of <code>premixed</code> and <code>ar</code>.

In [ ]:
# show the mixture temperatures
print(f'the premixed mixture temperature is {premixed.temperature:f} [K]')
print(f'the ar mixture temperature is       {ar.temperature:f} [K]')
print(f'the diluted mixture temperature is  {diluted.temperature:f} [K]')